**Experimental NB to train a DNN to classify Covid Images**

In [2]:
# common imports
import os
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from glob import glob
from pathlib import Path
from functools import partial

from sklearn.model_selection import train_test_split

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# prevent VRAM occupied
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

from keras.applications.resnet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

# Import functions for trianing the model
#%load_ext autoreload
#%autoreload 2
#import src.models.train_model as train_model

# load tensorboard extension
%reload_ext tensorboard

In [64]:
# Constants
SEED =42
IMAGE_SIZE = (224,224)
IMAGE_SHAPE = (224,224,3)
BATCH_SIZE = 20
SHUFFLE = True
TARGET_WIDTH= 224
TARGET_HEGIHT =224
NUM_CLASSES = 3 # number of ClassesNUM
NUM_EPOCHS = 20
learning_rate = 0.001    #initial training learning rate is moderately fast
ft_learning_rate = 1e-5 #fine tuning learning rate is slower
log_folder = 'logs' # logs folder

In [5]:
covid_image_train_data_gen = ImageDataGenerator(
    preprocessing_function= preprocess_input,
    validation_split=0.2,
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=True, #Boolean. Set each sample mean to 0.
    samplewise_std_normalization = False, #Boolean. Divide each input by its std.
    featurewise_std_normalization=False, # divide inputs by std of the dataset
    horizontal_flip = True, #Boolean. Randomly flip inputs horizontally.
    vertical_flip = False,  #Boolean. Randomly flip inputs vertically.
    zca_whitening=False,  # apply ZCA whitening
    height_shift_range= 0.05, #float: fraction of total height, if < 1, or pixels if >= 1.
    width_shift_range=0.1,  #float: fraction of total height, if < 1, or pixels if >= 1.
    rotation_range=20, #Int. Degree range for random rotations. 0 -180 degrees
    shear_range = 0.1, #Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees)
    fill_mode = 'nearest', #One of {"constant", "nearest", "reflect" or "wrap"}. Default is 'nearest'. 
    zoom_range=0.15) #Float or [lower, upper]. Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range]

covid_train_generator = covid_image_train_data_gen.flow_from_directory(
  '../data/raw/covid19-images',
  subset="training",
  class_mode='categorical',
  classes=['COVID19', 'NORMAL', 'PNEUMONIA'],
  seed=SEED,
  target_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE)

covid_valid_generator = covid_image_train_data_gen.flow_from_directory(
  '../data/raw/covid19-images',
  subset="validation",
  classes=['COVID19', 'NORMAL', 'PNEUMONIA'],
  class_mode='categorical',
  seed=SEED,
  target_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE)

**Build Model Using Transfer Learning**

In [65]:
#callback setup
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

#checkpoint_path = 'models/xray_class_weights.best.hdf5'
#checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1,
#                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor="val_loss", min_delta = 1e-4, patience = 10, mode = 'min',
                    restore_best_weights = True, verbose = 1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience = 3, min_delta=1e-4, min_lr=1e-6)#, verbose = 1,
                            #    min_delta = 1e-4, min_lr = 1e-6, mode = 'min', cooldown=1)

callbacks = [early, reduce_lr]

In [85]:
base_model2 = tf.keras.applications.ResNet50V2(input_shape=IMAGE_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
for layer in base_model2.layers:
            layer.trainable = False

In [86]:
#base_model.summary() #showed all layers frozen - check
#len(base_model.layers) #190 layers
head_model = base_model2.output
head_model = keras.layers.Flatten(name="flatten")(head_model)
#head_model = keras.layers.Dense(64, activation='relu')(head_model)   #Added Starts Here
#head_model = keras.layers.BatchNormalization()(head_model)
#head_model = keras.layers.Dropout(rate=0.3)(head_model)
#head_model = keras.layers.Dense(64, activation='relu')(head_model)
#head_model = keras.layers.BatchNormalization()(head_model)
#head_model = keras.layers.Dropout(rate=0.3)(head_model)
#head_model = keras.layers.Dense(32, activation='relu')(head_model)
#head_model = keras.layers.BatchNormalization()(head_model)
#head_model = keras.layers.Dropout(rate=0.3)(head_model)
#head_model = keras.layers.Dense(32, activation='relu')(head_model)
#head_model = keras.layers.BatchNormalization()(head_model)
#head_model = keras.layers.Dropout(rate=0.3)(head_model)                    #Added Ends Here
head_model = keras.layers.Dense(NUM_CLASSES,activation='softmax')(head_model)
model2 = keras.Model(inputs=base_model2.input, outputs=head_model)

In [87]:
#len(model.layers) #204
model2.summary()

In [88]:
optimizer=tf.keras.optimizers.Adam(
            learning_rate=learning_rate,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            amsgrad=True,
            name="Adam"
            )
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
model2.compile(optimizer=optimizer, loss = loss, metrics = metrics)

In [89]:
history = model2.fit(covid_train_generator,
                    epochs=NUM_EPOCHS,
                    validation_data=covid_train_generator,
                    steps_per_epoch = len(covid_train_generator)//BATCH_SIZE,#steps_per_epoch = 100,
                    validation_steps=len(covid_valid_generator)//BATCH_SIZE, #validation_steps= 25,
                    callbacks=callbacks)

In [90]:
for layer in base_model2.layers[186:]:
            layer.trainable = True

In [91]:
optimizer=tf.keras.optimizers.Adam(
            learning_rate=ft_learning_rate,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            amsgrad=True,
            name="Adam"
            )
model2.compile(optimizer=optimizer, loss = loss, metrics = metrics)

history_ft =  model2.fit(covid_train_generator,
                    epochs=10,
                    validation_data=covid_train_generator,
                    steps_per_epoch = len(covid_train_generator)//BATCH_SIZE,#steps_per_epoch = 100,
                    validation_steps=len(covid_valid_generator)//BATCH_SIZE, #validation_steps= 25,
                    callbacks=callbacks)

In [17]:
def plot_accuracy_and_loss(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,max(plt.ylim())])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [92]:
plot_accuracy_and_loss(history)

In [93]:
plot_accuracy_and_loss(history_ft)